In [2]:
import numpy as np
from math import sqrt
from pprint import pprint
from sklearn import datasets
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer                 # para crear metricas propias a mano
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics

In [3]:
# Cargar y explorar los datos
datos = datasets.load_diabetes()
# print(datos)
print(np.shape(datos.data))

# Extraer la matriz de datos "X" y la variable target "y"
X = datos.data
y = datos.target

# Partición EXTERNA: hold-out split 80% train - 20% test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarización de los datos de entrenamiento y test
standarizer = preprocessing.StandardScaler()

X_train = standarizer.fit_transform(X_train)
X_test = standarizer.transform(X_test)

(442, 10)


# SVR

- **C** float, default=1.0

    Regularization parameter. The strength of the regularization is inversely proportional to C. Must be strictly positive.     The penalty is a squared l2 penalty.


- **kernel** = {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'}

    Specifies the kernel type to be used in the algorithm. If none is given, ‘rbf’ will be used. If a callable is given it     is used to pre-compute the kernel matrix from data matrices; that matrix should be an array of shape 

- **degree** = int, default=3

    Degree of the polynomial kernel function (‘poly’). Must be non-negative. Ignored by all other kernels.

    
- **gamma** = {'scale', 'auto'} or float, default='scale'

    Kernel coefficient for 'rbf', 'poly' and 'sigmoid'.

    if gamma='scale' (default) is passed then it uses 1 / (n_features * X.var()) as value of gamma,

     if ‘auto’, uses 1 / n_features

     if float, must be non-negative.


In [5]:
from sklearn.model_selection import GridSearchCV

# hiperparametros
h_params = {'C': [0.01, 1, 10, 100],
            'kernel': ['linear', 'rbf', 'poly'],
            'gamma': ['auto', 'scale']}

# Métricas de evaluación.
metricas = ['r2', 'neg_root_mean_squared_error']

GD = GridSearchCV(estimator= SVR(),
                  param_grid= h_params,
                  scoring= metricas, 
                  cv= KFold(n_splits=5, shuffle= True, random_state= 42),
                  n_jobs= -1,
                  refit='r2')

GD.fit(X_train, y_train)

best_SVR = GD.best_estimator_
print(best_SVR)
print(GD.best_score_)

SVR(C=1, gamma='auto', kernel='linear')
0.47492607147800314


In [8]:
metricas = {
  'MAE' : lambda y, y_pred: metrics.mean_absolute_error(y, y_pred),
  'RMSE': lambda y, y_pred: sqrt(metrics.mean_squared_error(y, y_pred)),
  'MAPE': lambda y, y_pred: np.mean(np.abs((y - y_pred) / y)) * 100,
  'R2'  : lambda y, y_pred: metrics.r2_score(y, y_pred)}

y_pred = best_SVR.predict(X_test)

for nombre, metrica in metricas.items():
    print(f'para la metrica \t {nombre} \t  se obtienen un valor de \t  ---> \t {metrica(y_test, y_pred)}')

para la metrica 	 MAE 	  se obtienen un valor de 	  ---> 	 43.34160284347106
para la metrica 	 RMSE 	  se obtienen un valor de 	  ---> 	 54.220040394842165
para la metrica 	 MAPE 	  se obtienen un valor de 	  ---> 	 36.95619997812894
para la metrica 	 R2 	  se obtienen un valor de 	  ---> 	 0.44512484353785875
